In [1]:
from astroquery.gaia import Gaia
import pandas as pd

# Login

#### First let's login to Gaia archive. We can use the following formats:
#### Gaia.login_gui() - will display window to provide a username and password
#### Gaia.login() - a prompt will ask the user for name and password
#### Gaia.login(user=[username], password=[password]) - just type the username and password in the given fields
#### Gaia.login(credentials_file=[filename] - A path to the file that contains username and password. The username and password must be in different lines!

In [2]:
Gaia.login()

INFO: Login to gaia TAP server [astroquery.gaia.core]


User:  mmotylin
Password:  ················


OK
INFO: Login to gaia data server [astroquery.gaia.core]
OK


# Prepare Exoplanets data for crossmatching query.
### Remove any targets that do not have GAIA ID. Extract numerical value of GAIA ID and save it as integer. Remove duplicates so that we end up with a single instance of each host star.

In [37]:
df = pd.read_csv("data/initial_datasets/PSCompPars_2022.11.10_06.54.37.csv", skiprows=9)
df.dropna(subset=["gaia_id"], inplace=True)
df["source_id"] = df["gaia_id"].str.rsplit(" ", n=1, expand=True)[1].dropna().astype("int64")
df["Host"] = df["hostname"].str.replace(" ", "")
df.drop_duplicates(subset=["Host"], inplace=True)
df.drop(["gaia_id", "hostname"], axis=1, inplace=True)
df.to_csv("data/initial_datasets/NASA_ex_cleaned.csv", index=False)

# Upload table with Gaia ID's from NASA Exoplanets Archive
### Can be done on the Gaia archive website using GUI. Cannot upload the same table twice unless we change the name as they are unique.

In [41]:
#job = Gaia.delete_user_table("exoplanets")
Gaia.upload_table(upload_resource="data/initial_datasets/NASA_ex_cleaned.csv", table_name="exoplanets", format="CSV")

Sending file: data/initial_datasets/NASA_ex_cleaned.csv
Uploaded table 'exoplanets'.


# Submit crossmatch query

##### Provide gaia table name to query as well as your username that will be used to load the uploaded table.
##### Set a name for the file to which queried data will be saved.

In [42]:
table = "gaiadr3.dr2_neighbourhood"
username = "mmotylin"
filename = "exoplanets_query.csv"

In [51]:
query = f"""
SELECT exoplanets.*, both_names.dr2_source_id, both_names.dr3_source_id
FROM user_{username}.exoplanets AS exoplanets
JOIN {table} as both_names 
    ON both_names.dr2_source_id = exoplanets.source_id
WHERE ABS(magnitude_difference) < 0.1
"""

In [52]:
Gaia.launch_job_async(query).get_results().to_pandas().to_csv(f"data/initial_datasets/{filename}", index=False)

INFO: Query finished. [astroquery.utils.tap.core]


# Clean up after query.

In [85]:
df1 = pd.read_csv(f"data/initial_datasets/{filename}")
df1["source_id"] = df1["dr3_source_id"]
df1 = df1[["pl_name", "source_id", "host"]]
df1.to_csv("data/initial_datasets/exoplanets.csv", index=False)